In [58]:
import os
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta

import pandas as pd
from pandas.tseries.offsets import QuarterEnd, YearEnd

import holidays

# Initialize a dataframe with date range

In [59]:
start_period = '2000-01-01'
end_period = '2029-12-31'

date_list = pd.date_range(start=start_period, end=end_period).to_pydatetime().tolist()
df = pd.DataFrame(data=date_list, columns=['date_id'])
df.shape

(10958, 1)

In [60]:
df.head()

,date_id
0,2000-01-01
1,2000-01-02
2,2000-01-03
3,2000-01-04
4,2000-01-05


# Enrich it with datetime attributes

In [61]:
%%time 

# init a list of holidays for every country, it will fill after the first request
us_holidays = holidays.US()
ua_holidays = holidays.UA()

# set prefixies
calendar = 'calendar'
fiscal = 'fiscal'

# basic attributes
df['month_name'] = df['date_id'].dt.strftime("%B")
df['weekday_name'] = df['date_id'].dt.strftime("%A")
df['day_of_month'] = df['date_id'].dt.day
df['day_of_year'] = df['date_id'].dt.day_of_year  # another approach .apply(lambda date: date.timetuple().tm_yday)
df['week_of_year'] = df['date_id'].dt.isocalendar().week
df['day_of_week'] = df['date_id'].dt.weekday + 1 # add 1 for humans:)
df['days_in_month'] = df['date_id'].dt.days_in_month

# boolean attributes
df['is_end_of_month'] = df['date_id'].apply(lambda date: True if pd.Period(date, freq='M').end_time.date() == date else False)
df['is_weekend'] = df['day_of_week'].apply(lambda weekday: True if weekday > 5 else False)
df['is_leap_year'] = df['date_id'].dt.is_leap_year  # another approach .apply(calendar.isleap)
df['is_us_holiday'] = df['date_id'].apply(lambda date: date in us_holidays)
df['is_ua_holiday'] = df['date_id'].apply(lambda date: date in ua_holidays)
df['us_holiday_name'] = df['date_id'].map(us_holidays)
df['ua_holiday_name'] = df['date_id'].map(ua_holidays) # TODO - translate in English on-the-fly?

# calendar year attributes
df[f'{calendar}_month'] = df['date_id'].dt.month
df[f'{calendar}_quarter'] = df['date_id'].dt.quarter
df[f'{calendar}_year'] = df['date_id'].dt.year
df[f'{calendar}_is_eoq'] = df['date_id'].apply(lambda date: True if date == (date - QuarterEnd(startingMonth=0, n=0)).date() else False)
df[f'{calendar}_is_eoy'] = df['date_id'].apply(lambda date: True if date == (date - YearEnd(n=0)).date() else False)
df[f'{calendar}_yyyy_qq'] = df['date_id'].dt.to_period('Q').dt.strftime('%F-Q%q')
df[f'{calendar}_yyyy_mm'] = df['date_id'].dt.strftime('%Y-%m')
df[f'{calendar}_yyyy_mon'] = df['date_id'].dt.strftime('%Y-%b')

# fiscal year attributes
df[f'{fiscal}_month'] = df['date_id'].apply(lambda date: (date - relativedelta(months=1)).month)
df[f'{fiscal}_quarter'] = df['date_id'].dt.to_period('Q-FEB') # use as a RAW source for futher extractions
df[f'{fiscal}_year'] =  df[f'{fiscal}_quarter'].dt.qyear
df[f'{fiscal}_is_eoq'] = df['date_id'].apply(lambda date: True if date == (date - QuarterEnd(startingMonth=1, n=0)).date() else False) # set "startingMonth" arg with offset
df[f'{fiscal}_is_eoy'] = df[[f'{calendar}_month','day_of_month']].apply(lambda row: True if row[0] == 1 and row[1] == 31 else False, axis=1)
df[f'{fiscal}_yyyy_qq'] = df[f'{fiscal}_quarter'].dt.strftime('F%F-Q%q')
df[f'{fiscal}_yyyy_mm'] = pd.to_datetime(df[f'{fiscal}_year'].astype(str) + '-' + df[f'{fiscal}_month'].astype(str)) # # use as a RAW source for futher extractions
df[f'{fiscal}_yyyy_mon'] = df[f'{fiscal}_yyyy_mm'].dt.strftime('F%Y-%b')

# reassign these columns with the values in a proper format
df[f'{fiscal}_yyyy_mm'] = df[f'{fiscal}_yyyy_mm'].dt.strftime('F%Y-%m')
df[f'{fiscal}_quarter'] = df[f'{fiscal}_quarter'].dt.quarter

df.head(31)

CPU times: user 3.83 s, sys: 7.93 ms, total: 3.83 s
Wall time: 3.83 s


,date_id,month_name,weekday_name,day_of_month,day_of_year,week_of_year,day_of_week,days_in_month,is_end_of_month,is_weekend,is_leap_year,is_us_holiday,is_ua_holiday,us_holiday_name,ua_holiday_name,calendar_month,calendar_quarter,calendar_year,calendar_is_eoq,calendar_is_eoy,calendar_yyyy_qq,calendar_yyyy_mm,calendar_yyyy_mon,fiscal_month,fiscal_quarter,fiscal_year,fiscal_is_eoq,fiscal_is_eoy,fiscal_yyyy_qq,fiscal_yyyy_mm,fiscal_yyyy_mon
0,2000-01-01,January,Saturday,1,1,52,6,31,False,True,True,True,True,New Year's Day,Новий рік,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
1,2000-01-02,January,Sunday,2,2,52,7,31,False,True,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
2,2000-01-03,January,Monday,3,3,1,1,31,False,False,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
3,2000-01-04,January,Tuesday,4,4,1,2,31,False,False,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
4,2000-01-05,January,Wednesday,5,5,1,3,31,False,False,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
5,2000-01-06,January,Thursday,6,6,1,4,31,False,False,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
6,2000-01-07,January,Friday,7,7,1,5,31,False,False,True,False,True,NaN,Різдво Христове (православне),1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
7,2000-01-08,January,Saturday,8,8,1,6,31,False,True,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
8,2000-01-09,January,Sunday,9,9,1,7,31,False,True,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec
9,2000-01-10,January,Monday,10,10,2,1,31,False,False,True,False,False,NaN,NaN,1,1,2000,False,False,2000-Q1,2000-01,2000-Jan,12,4,2000,False,False,F2000-Q4,F2000-12,F2000-Dec


In [62]:
df.tail()

,date_id,month_name,weekday_name,day_of_month,day_of_year,week_of_year,day_of_week,days_in_month,is_end_of_month,is_weekend,is_leap_year,is_us_holiday,is_ua_holiday,us_holiday_name,ua_holiday_name,calendar_month,calendar_quarter,calendar_year,calendar_is_eoq,calendar_is_eoy,calendar_yyyy_qq,calendar_yyyy_mm,calendar_yyyy_mon,fiscal_month,fiscal_quarter,fiscal_year,fiscal_is_eoq,fiscal_is_eoy,fiscal_yyyy_qq,fiscal_yyyy_mm,fiscal_yyyy_mon
10953,2029-12-27,December,Thursday,27,361,52,4,31,False,False,False,False,False,NaN,NaN,12,4,2029,False,False,2029-Q4,2029-12,2029-Dec,11,4,2030,False,False,F2030-Q4,F2030-11,F2030-Nov
10954,2029-12-28,December,Friday,28,362,52,5,31,False,False,False,False,False,NaN,NaN,12,4,2029,False,False,2029-Q4,2029-12,2029-Dec,11,4,2030,False,False,F2030-Q4,F2030-11,F2030-Nov
10955,2029-12-29,December,Saturday,29,363,52,6,31,False,True,False,False,False,NaN,NaN,12,4,2029,False,False,2029-Q4,2029-12,2029-Dec,11,4,2030,False,False,F2030-Q4,F2030-11,F2030-Nov
10956,2029-12-30,December,Sunday,30,364,52,7,31,False,True,False,False,False,NaN,NaN,12,4,2029,False,False,2029-Q4,2029-12,2029-Dec,11,4,2030,False,False,F2030-Q4,F2030-11,F2030-Nov
10957,2029-12-31,December,Monday,31,365,1,1,31,True,False,False,False,False,NaN,NaN,12,4,2029,True,True,2029-Q4,2029-12,2029-Dec,11,4,2030,False,False,F2030-Q4,F2030-11,F2030-Nov


In [63]:
df.sample(10)

,date_id,month_name,weekday_name,day_of_month,day_of_year,week_of_year,day_of_week,days_in_month,is_end_of_month,is_weekend,is_leap_year,is_us_holiday,is_ua_holiday,us_holiday_name,ua_holiday_name,calendar_month,calendar_quarter,calendar_year,calendar_is_eoq,calendar_is_eoy,calendar_yyyy_qq,calendar_yyyy_mm,calendar_yyyy_mon,fiscal_month,fiscal_quarter,fiscal_year,fiscal_is_eoq,fiscal_is_eoy,fiscal_yyyy_qq,fiscal_yyyy_mm,fiscal_yyyy_mon
8309,2022-10-01,October,Saturday,1,274,39,6,31,False,True,False,False,False,NaN,NaN,10,4,2022,False,False,2022-Q4,2022-10,2022-Oct,9,3,2023,False,False,F2023-Q3,F2023-09,F2023-Sep
4153,2011-05-16,May,Monday,16,136,20,1,31,False,False,False,False,False,NaN,NaN,5,2,2011,False,False,2011-Q2,2011-05,2011-May,4,1,2012,False,False,F2012-Q1,F2012-04,F2012-Apr
4990,2013-08-30,August,Friday,30,242,35,5,31,False,False,False,False,False,NaN,NaN,8,3,2013,False,False,2013-Q3,2013-08,2013-Aug,7,2,2014,False,False,F2014-Q2,F2014-07,F2014-Jul
2725,2007-06-18,June,Monday,18,169,25,1,30,False,False,False,False,False,NaN,NaN,6,2,2007,False,False,2007-Q2,2007-06,2007-Jun,5,2,2008,False,False,F2008-Q2,F2008-05,F2008-May
10747,2029-06-04,June,Monday,4,155,23,1,30,False,False,False,False,False,NaN,NaN,6,2,2029,False,False,2029-Q2,2029-06,2029-Jun,5,2,2030,False,False,F2030-Q2,F2030-05,F2030-May
767,2002-02-06,February,Wednesday,6,37,6,3,28,False,False,False,False,False,NaN,NaN,2,1,2002,False,False,2002-Q1,2002-02,2002-Feb,1,4,2002,False,False,F2002-Q4,F2002-01,F2002-Jan
483,2001-04-28,April,Saturday,28,118,17,6,30,False,True,False,False,False,NaN,NaN,4,2,2001,False,False,2001-Q2,2001-04,2001-Apr,3,1,2002,False,False,F2002-Q1,F2002-03,F2002-Mar
4630,2012-09-04,September,Tuesday,4,248,36,2,30,False,False,True,False,False,NaN,NaN,9,3,2012,False,False,2012-Q3,2012-09,2012-Sep,8,3,2013,False,False,F2013-Q3,F2013-08,F2013-Aug
4037,2011-01-20,January,Thursday,20,20,3,4,31,False,False,False,False,False,NaN,NaN,1,1,2011,False,False,2011-Q1,2011-01,2011-Jan,12,4,2011,False,False,F2011-Q4,F2011-12,F2011-Dec
8387,2022-12-18,December,Sunday,18,352,50,7,31,False,True,False,False,False,NaN,NaN,12,4,2022,False,False,2022-Q4,2022-12,2022-Dec,11,4,2023,False,False,F2023-Q4,F2023-11,F2023-Nov


# SAVE TO CSV

In [66]:
dataset_dir = '../datasets'
os.makedirs(dataset_dir)
df.to_csv(f'{dataset_dir}/dim_dates.csv', index=False)